In [27]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt


from tqdm import tqdm
from nba_api.stats.static import players, teams
from nba_api.stats.library.parameters import SeasonAll

In [28]:
pd.options.display.max_columns=200

## Compiling and Cleaning Data

In [77]:
team_gamelog = pd.read_csv('./data/gamelog_2018-19.csv')
player_gamelog = pd.read_csv('./data/player_gamelogs_2010-11.csv')

In [82]:
team_gamelog

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,FT_PCT_RANK,OREB_RANK,DREB_RANK,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK
0,2018-19,1610612751,BKN,Brooklyn Nets,21801221,2019-04-10T00:00:00,BKN vs. MIA,W,48.0,43,114,0.377,21,58,0.362,6,7,0.857,20,48,68,29,12.0,7,3,4,13,12,113,19.0,1,1,1,1,135,760,12,2330,23,8,1130,2408,2440,451,23,20,3,410,636,1203,1731,731,40,2422,1031,226
1,2018-19,1610612766,CHA,Charlotte Hornets,21801222,2019-04-10T00:00:00,CHA vs. ORL,L,48.0,41,78,0.526,8,29,0.276,24,27,0.889,4,28,32,21,8.0,3,4,2,14,19,114,-8.0,1,1231,1231,1231,135,1130,2303,300,1907,1510,2022,331,661,257,2360,2150,2400,1752,96,2301,1329,146,80,1522,959,1711
2,2018-19,1610612744,GSW,Golden State Warriors,21801225,2019-04-10T00:00:00,GSW @ MEM,L,48.0,46,92,0.500,13,30,0.433,12,14,0.857,9,30,39,32,14.0,3,5,0,17,18,117,-15.0,1,1231,1231,1231,135,360,755,535,646,1381,427,1967,2157,451,1362,1905,1966,175,1135,2301,917,1,376,1725,733,2080
3,2018-19,1610612743,DEN,Denver Nuggets,21801228,2019-04-10T00:00:00,DEN vs. MIN,W,48.0,39,87,0.448,10,33,0.303,11,17,0.647,12,41,53,23,13.0,6,4,0,12,22,99,4.0,1,1,1,1,135,1496,1438,1448,1372,972,1785,2083,1894,2154,649,288,267,1394,890,1543,1329,1,23,823,2012,977
4,2018-19,1610612755,PHI,Philadelphia 76ers,21801224,2019-04-10T00:00:00,PHI vs. CHI,W,48.0,52,93,0.559,12,26,0.462,9,15,0.600,9,38,47,30,7.0,4,10,3,11,13,125,16.0,1,1,1,1,135,53,626,94,864,1883,267,2271,2083,2311,1362,610,893,308,35,2133,53,393,11,2382,315,303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,2018-19,1610612752,NYK,New York Knicks,21800007,2018-10-17T00:00:00,NYK vs. ATL,W,48.0,45,99,0.455,12,33,0.364,24,31,0.774,10,36,46,21,16.0,12,6,3,23,30,126,19.0,1,1,1,1,135,478,189,1324,864,972,1076,331,330,1192,1108,921,1019,1752,1614,149,603,393,1639,42,278,226
2456,2018-19,1610612744,GSW,Golden State Warriors,21800002,2018-10-16T00:00:00,GSW vs. OKC,W,48.0,42,95,0.442,7,26,0.269,17,18,0.944,17,41,58,28,21.0,7,7,6,29,21,108,8.0,1,1,1,1,135,929,432,1558,2102,1883,2070,1201,1774,75,92,288,63,535,2319,1203,371,1545,2347,1044,1439,674
2457,2018-19,1610612760,OKC,Oklahoma City Thunder,21800002,2018-10-16T00:00:00,OKC @ GSW,L,48.0,33,91,0.363,10,37,0.270,24,37,0.649,16,29,45,21,15.0,12,6,7,21,29,100,-8.0,1,1231,1231,1231,135,2292,896,2389,1372,534,2063,331,92,2153,159,2035,1165,1752,1390,149,603,1859,1167,73,1961,1711
2458,2018-19,1610612738,BOS,Boston Celtics,21800001,2018-10-16T00:00:00,BOS vs. PHI,W,48.0,42,97,0.433,11,37,0.297,10,14,0.714,12,43,55,21,15.0,7,5,5,20,20,105,18.0,1,1,1,1,135,929,293,1712,1110,534,1828,2177,2157,1707,649,151,155,1752,1390,1203,917,1133,940,1295,1659,251


In [83]:
player_gamelog

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,NBA_FANTASY_PTS,DD2,TD3,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,FT_PCT_RANK,OREB_RANK,DREB_RANK,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK
0,2010-11,1894,Corey Maggette,1610612749,MIL,Milwaukee Bucks,21001225,2011-04-13T00:00:00,MIL @ OKC,W,18.776667,3,5,0.600,0,0,0.000,6,6,1.000,0,1,1,2,2,1,0,0,2,3,12,2,17.2,0,0,1,1,1,1,16358,11210,15311,4885,8666,13752,8666,1409,2309,1,13200,16337,18587,7589,4475,4411,7858,1,10314,5059,8001,9453,13123,2133,38
1,2010-11,101127,Jarrett Jack,1610612740,NOH,New Orleans Hornets,21001223,2011-04-13T00:00:00,NOH @ DAL,L,28.658333,8,15,0.533,1,1,1.000,5,6,0.833,0,0,0,7,1,0,1,0,2,4,22,-22,34.5,0,0,1,12551,12551,12551,9358,1890,2643,7507,4260,10294,1,2148,2309,7094,13200,21028,22216,1199,9152,11571,2612,1,10314,3087,1928,24740,4046,2133,38
2,2010-11,200770,Jordan Farmar,1610612751,NJN,New Jersey Nets,21001222,2011-04-13T00:00:00,NJN @ CHI,L,38.166667,7,15,0.467,2,6,0.333,5,6,0.833,0,4,4,12,4,0,2,0,1,4,21,7,45.8,1,0,1,12551,12551,12551,2634,2971,2643,11367,1830,1129,5649,2148,2309,7094,13200,5857,8902,195,843,11571,928,1,4451,3087,2291,5411,1206,1,38
3,2010-11,2405,Amar'e Stoudemire,1610612752,NYK,New York Knicks,21001216,2011-04-13T00:00:00,NYK @ BOS,L,20.450000,6,15,0.400,0,0,0.000,2,3,0.667,2,2,4,1,2,0,1,0,1,4,14,-3,21.3,0,0,1,12551,12551,12551,15235,4364,2643,13736,8666,13752,8666,6796,6990,9548,3434,12039,8902,11552,4475,11571,2612,1,4451,3087,6325,15016,10517,2133,38
4,2010-11,201957,Byron Mullens,1610612760,OKC,Oklahoma City Thunder,21001225,2011-04-13T00:00:00,OKC vs. MIL,L,29.178333,4,9,0.444,0,0,0.000,2,4,0.500,1,4,5,0,2,1,0,1,2,2,10,-22,17.0,0,0,1,12551,12551,12551,8979,8488,8868,12111,8666,13752,8666,6796,4496,10901,6688,5857,6752,17183,4475,4411,7858,16556,10314,7975,9880,24740,13247,2133,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25148,2010-11,2547,Chris Bosh,1610612748,MIA,Miami Heat,21000001,2010-10-26T00:00:00,MIA @ BOS,L,37.616667,3,11,0.273,0,0,0.000,2,2,1.000,2,6,8,2,1,1,1,1,4,2,8,-17,25.6,0,0,1,12551,12551,12551,2971,11210,6221,18827,8666,13752,8666,6796,8399,1,3434,2657,2794,7589,9152,4411,2612,16556,20677,7975,12071,23950,8031,2133,38
25149,2010-11,979,Jermaine O'Neal,1610612738,BOS,Boston Celtics,21000001,2010-10-26T00:00:00,BOS vs. MIA,W,12.783333,0,3,0.000,0,0,0.000,1,2,0.500,1,1,2,0,1,0,2,0,1,2,1,1,8.4,0,0,1,1,1,1,20149,21564,18895,21564,8666,13752,8666,10775,8399,10901,6688,16337,14958,17183,9152,11571,928,1,4451,7975,22002,10499,19183,2133,38
25150,2010-11,202083,Wesley Matthews,1610612757,POR,Portland Trail Blazers,21000002,2010-10-26T00:00:00,POR vs. PHX,W,30.400000,5,12,0.417,1,2,0.500,2,3,0.667,2,4,6,3,0,0,0,1,4,6,13,22,24.7,0,0,1,1,1,1,8098,6173,5171,13402,4260,7554,2796,6796,6990,9548,3434,5857,5066,5151,16494,11571,7858,16556,20677,1010,7152,499,8499,2133,38
25151,2010-11,689,Theo Ratliff,1610612747,LAL,Los Angeles Lakers,21000003,2010-10-26T00:00:00,LAL vs. HOU,W,16.883333,0,2,0.000,0,0,0.000,0,0,0.000,1,1,2,1,0,0,2,1,1,2,0,-4,9.9,0,0,1,1,1,1,17640,21564,20651,21564,8666,13752,8666,14187,15157,14187,6688,16337,14958,11552,16494,11571,928,16556,4451,7975,22308,16054,18163,2133,38


In [31]:
def clean_team_gamelog(team_gamelog = team_gamelog):
    df = team_gamelog.copy()
    df = df[['SEASON_YEAR', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M',
       'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
       'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS']]
    
    # Convert GAME_DATE to datetime so it is sortable
    df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])
    
    # Sort by team and date
    df.sort_values(['TEAM_ABBREVIATION', 'GAME_DATE'], inplace=True)
    
    # Combine rows so that one instance shows a game with team stats and opponent stats
    full_gamelogs = pd.merge(df, df, how='inner', on='GAME_ID', suffixes=['', '_opp'])

    # Remove rows that were created where the team is playing itself
    full_gamelogs = full_gamelogs.loc[full_gamelogs['TEAM_ABBREVIATION'] != full_gamelogs['TEAM_ABBREVIATION_opp']]

    # Add column for HOME_GAME
    full_gamelogs['HOME_GAME'] = (full_gamelogs['MATCHUP'].str[:6] == (full_gamelogs['TEAM_ABBREVIATION'].str[:] + ' vs')).astype(int)
    
    full_gamelogs['WIN'] = (full_gamelogs['WL'] == 'W').astype(int)
    full_gamelogs.drop(columns='WL', inplace=True)
    
    full_gamelogs['point_diff'] = full_gamelogs['PLUS_MINUS']
    
    full_gamelogs.sort_values(['TEAM_ABBREVIATION', 'GAME_DATE'], inplace=True)
    
    full_gamelogs['prev_WIN'] = full_gamelogs['WIN'].shift(1)
    full_gamelogs['prev2_WIN'] = full_gamelogs['WIN'].shift(2)
    
    # Linearly dependent variables
    full_gamelogs = full_gamelogs.drop(columns=['FG_PCT', 'FG3_PCT', 'FT_PCT', 'REB',
                                               'FG_PCT_opp', 'FG3_PCT_opp', 'FT_PCT_opp', 'REB_opp',
                                               'BLK_opp', 'BLKA_opp', 'PF_opp', 'PFD_opp', 'PLUS_MINUS_opp',
                                               'MIN_opp'])
    
    return full_gamelogs
    
clean_gamelogs_2018 = clean_team_gamelog(team_gamelog)
clean_gamelogs_2018

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,MIN,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,SEASON_YEAR_opp,TEAM_ID_opp,TEAM_ABBREVIATION_opp,TEAM_NAME_opp,GAME_DATE_opp,MATCHUP_opp,WL_opp,FGM_opp,FGA_opp,FG3M_opp,FG3A_opp,FTM_opp,FTA_opp,OREB_opp,DREB_opp,AST_opp,TOV_opp,STL_opp,PTS_opp,HOME_GAME,WIN,point_diff,prev_WIN,prev2_WIN
1,2018-19,1610612737,ATL,Atlanta Hawks,21800007,2018-10-17,ATL @ NYK,48.0,41,90,10,36,15,18,7,38,19,24.0,7,3,6,30,23,107,-19.0,2018-19,1610612752,NYK,New York Knicks,2018-10-17,NYK vs. ATL,W,45,99,12,33,24,31,10,36,21,16.0,12,126,0,0,-19.0,NaN,NaN
5,2018-19,1610612737,ATL,Atlanta Hawks,21800020,2018-10-19,ATL @ MEM,48.0,41,83,14,30,21,36,11,29,27,19.0,9,5,4,31,29,117,-14.0,2018-19,1610612763,MEM,Memphis Grizzlies,2018-10-19,MEM vs. ATL,W,44,81,13,26,30,43,7,34,32,17.0,9,131,0,0,-14.0,0.0,NaN
9,2018-19,1610612737,ATL,Atlanta Hawks,21800036,2018-10-21,ATL @ CLE,48.0,48,98,22,47,15,19,9,35,32,12.0,9,7,2,21,25,133,22.0,2018-19,1610612739,CLE,Cleveland Cavaliers,2018-10-21,CLE vs. ATL,L,46,99,10,27,9,14,15,34,23,18.0,5,111,0,1,22.0,0.0,0.0
13,2018-19,1610612737,ATL,Atlanta Hawks,21800052,2018-10-24,ATL vs. DAL,48.0,37,91,15,38,22,31,10,44,28,15.0,6,6,9,26,27,111,7.0,2018-19,1610612742,DAL,Dallas Mavericks,2018-10-24,DAL @ ATL,L,38,91,12,40,16,26,9,43,25,17.0,7,104,1,1,7.0,1.0,0.0
17,2018-19,1610612737,ATL,Atlanta Hawks,21800076,2018-10-27,ATL vs. CHI,48.0,27,84,12,40,19,23,11,37,20,22.0,9,8,6,13,21,85,-12.0,2018-19,1610612741,CHI,Chicago Bulls,2018-10-27,CHI @ ATL,W,36,83,12,33,13,16,8,44,25,23.0,14,97,1,0,-12.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266,2018-19,1610612764,WAS,Washington Wizards,21801148,2019-03-31,WAS @ DEN,48.0,37,92,6,24,15,20,9,45,24,12.0,9,6,5,21,18,95,5.0,2018-19,1610612743,DEN,Denver Nuggets,2019-03-31,DEN vs. WAS,L,35,93,7,36,13,20,12,45,18,11.0,6,90,0,1,5.0,0.0,1.0
1478,2018-19,1610612764,WAS,Washington Wizards,21801167,2019-04-03,WAS vs. CHI,48.0,45,97,5,27,19,26,9,30,24,12.0,8,9,5,18,23,114,-1.0,2018-19,1610612741,CHI,Chicago Bulls,2019-04-03,CHI @ WAS,W,48,90,5,19,14,17,9,40,22,20.0,7,115,1,0,-1.0,1.0,0.0
4886,2018-19,1610612764,WAS,Washington Wizards,21801182,2019-04-05,WAS vs. SAS,48.0,42,88,9,32,19,22,9,25,22,10.0,2,4,4,16,18,112,-17.0,2018-19,1610612759,SAS,San Antonio Spurs,2019-04-05,SAS @ WAS,W,51,91,10,25,17,20,12,30,34,9.0,3,129,1,0,-17.0,0.0,1.0
4414,2018-19,1610612764,WAS,Washington Wizards,21801204,2019-04-07,WAS @ NYK,48.0,43,103,10,34,14,18,21,29,24,9.0,12,4,9,16,14,110,-3.0,2018-19,1610612752,NYK,New York Knicks,2019-04-07,NYK vs. WAS,W,41,80,17,34,14,19,7,36,24,15.0,5,113,0,0,-3.0,0.0,0.0


In [32]:
# def get_cumulative_stats(df, n=5):
#     df = df.copy()
#     stats = ['MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M',
#        'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
#        'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS',
#        'FGM_opp', 'FGA_opp', 'FG_PCT_opp', 'FG3M_opp', 'FG3A_opp',
#        'FG3_PCT_opp', 'FTM_opp', 'FTA_opp', 'FT_PCT_opp', 'OREB_opp',
#        'DREB_opp', 'REB_opp', 'AST_opp', 'TOV_opp', 'STL_opp', 'BLK_opp',
#        'BLKA_opp', 'PF_opp', 'PFD_opp', 'PTS_opp', 'PLUS_MINUS_opp',
#        'WIN']
#     all_teams_cume_stats = []
    
#     team_abbreviations = df['TEAM_ABBREVIATION'].unique()
#     for team in team_abbreviations:
#         team_df = df.loc[df['TEAM_ABBREVIATION'] == team].sort_values(['GAME_DATE'])        
#         team_df[stats] = team_df[stats].shift(1).expanding(min_periods=5).mean()
#         last_n_games = df.loc[df['TEAM_ABBREVIATION'] == team].sort_values(['GAME_DATE'])    

#         last_n_games[stats] = last_n_games[stats].shift(1).rolling(n, min_periods=5).mean()
#         combined_df = pd.merge(team_df, last_n_games[stats], left_index=True, right_index=True, suffixes =['_avg', '_last{}'.format(n)])
#         combined_df.sort_values(['GAME_DATE'], inplace=True)
#         all_teams_cume_stats.append(combined_df)
        
#     full_df = pd.concat(all_teams_cume_stats)
    
#     return full_df


# full_gamelogs_cume = get_cumulative_stats(clean_gamelogs_2018)
# full_gamelogs_cume
        

KeyError: "['PLUS_MINUS_opp', 'FT_PCT', 'BLKA_opp', 'FG3_PCT_opp', 'FT_PCT_opp', 'REB_opp', 'REB', 'FG_PCT', 'FG3_PCT', 'PF_opp', 'PFD_opp', 'FG_PCT_opp', 'BLK_opp'] not in index"

In [72]:
def get_weighted_cume_stats(df, n=10):
    df = df.copy()
    stats = ['MIN', 'FGM', 'FGA', 'FG3M',
             'FG3A', 'FTM', 'FTA', 'OREB', 'DREB', 'AST',
             'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS',
             'PLUS_MINUS', 'FGM_opp', 'FGA_opp', 'FG3M_opp',
             'FG3A_opp', 'FTM_opp', 'FTA_opp', 'OREB_opp', 
             'DREB_opp', 'AST_opp', 'TOV_opp', 'STL_opp', 
             'PTS_opp', 'WIN']
    all_teams_cume_stats = []
    
    team_abbreviations = df['TEAM_ABBREVIATION'].unique()
    
    weights = np.array([0.05, 0.05, 0.05, 0.05, 0.05,
                       0.15, 0.15, 0.15, 0.15, 0.15])
    
    sum_weights = np.sum(weights)
    
    for team in team_abbreviations:
        team_df = df.loc[df['TEAM_ABBREVIATION'] == team].sort_values(['GAME_DATE'])        
        team_df[stats] = (team_df[stats].shift(1).rolling(window=n)
                         .apply(lambda row: np.sum(weights*row) / sum_weights, raw=False))
                        
        
        
        team_df.sort_values(['GAME_DATE'], inplace=True)
        all_teams_cume_stats.append(team_df)
        
    full_df = pd.concat(all_teams_cume_stats)
    
    return full_df

weighted_stats_2018 = get_weighted_cume_stats(clean_gamelogs_2018)
weighted_stats_2018

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,MIN,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,SEASON_YEAR_opp,TEAM_ID_opp,TEAM_ABBREVIATION_opp,TEAM_NAME_opp,GAME_DATE_opp,MATCHUP_opp,WL_opp,FGM_opp,FGA_opp,FG3M_opp,FG3A_opp,FTM_opp,FTA_opp,OREB_opp,DREB_opp,AST_opp,TOV_opp,STL_opp,PTS_opp,HOME_GAME,WIN,point_diff,prev_WIN,prev2_WIN
1,2018-19,1610612737,ATL,Atlanta Hawks,21800007,2018-10-17,ATL @ NYK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,1610612752,NYK,New York Knicks,2018-10-17,NYK vs. ATL,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,-19.0,NaN,NaN
5,2018-19,1610612737,ATL,Atlanta Hawks,21800020,2018-10-19,ATL @ MEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,1610612763,MEM,Memphis Grizzlies,2018-10-19,MEM vs. ATL,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,-14.0,0.0,NaN
9,2018-19,1610612737,ATL,Atlanta Hawks,21800036,2018-10-21,ATL @ CLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,1610612739,CLE,Cleveland Cavaliers,2018-10-21,CLE vs. ATL,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,22.0,0.0,0.0
13,2018-19,1610612737,ATL,Atlanta Hawks,21800052,2018-10-24,ATL vs. DAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,1610612742,DAL,Dallas Mavericks,2018-10-24,DAL @ ATL,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,7.0,1.0,0.0
17,2018-19,1610612737,ATL,Atlanta Hawks,21800076,2018-10-27,ATL vs. CHI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,1610612741,CHI,Chicago Bulls,2018-10-27,CHI @ ATL,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,-12.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266,2018-19,1610612764,WAS,Washington Wizards,21801148,2019-03-31,WAS @ DEN,48.25,41.75,89.40,10.05,31.05,19.95,26.35,11.95,32.65,23.95,13.15,7.15,3.35,6.10,18.20,21.90,113.50,-5.15,2018-19,1610612743,DEN,Denver Nuggets,2019-03-31,DEN vs. WAS,L,45.75,89.60,12.95,33.50,14.20,18.05,10.75,34.60,28.75,14.00,7.80,118.65,0,0.25,5.0,0.0,1.0
1478,2018-19,1610612764,WAS,Washington Wizards,21801167,2019-04-03,WAS vs. CHI,48.25,41.20,90.80,9.75,30.50,19.80,26.65,12.20,34.40,24.05,13.30,7.85,3.95,6.30,19.20,21.85,111.95,-4.40,2018-19,1610612741,CHI,Chicago Bulls,2019-04-03,CHI @ WAS,W,44.65,90.35,12.00,34.25,15.05,19.50,11.05,36.45,27.15,14.30,7.65,116.35,1,0.35,-1.0,1.0,0.0
4886,2018-19,1610612764,WAS,Washington Wizards,21801182,2019-04-05,WAS vs. SAS,48.25,41.80,91.55,8.95,29.85,20.20,27.30,11.05,33.85,23.90,13.05,8.15,4.85,5.70,19.20,22.15,112.75,-3.75,2018-19,1610612759,SAS,San Antonio Spurs,2019-04-05,SAS @ WAS,W,45.15,90.00,10.85,31.65,15.35,19.55,10.45,37.70,26.65,15.50,7.65,116.50,1,0.35,-17.0,0.0,1.0
4414,2018-19,1610612764,WAS,Washington Wizards,21801204,2019-04-07,WAS @ NYK,48.25,42.15,91.70,8.85,29.70,19.05,25.75,10.85,33.00,23.80,12.20,7.65,4.70,5.65,18.55,21.40,112.20,-4.85,2018-19,1610612752,NYK,New York Knicks,2019-04-07,NYK vs. WAS,W,45.60,90.25,10.65,30.45,15.20,18.95,10.60,36.65,26.30,14.45,6.95,117.05,0,0.30,-3.0,0.0,0.0


In [35]:
weighted_stats_2018.loc[weighted_stats_2018['TEAM_ABBREVIATION'] == 'ATL'].head(15)

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,MIN,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,SEASON_YEAR_opp,TEAM_ID_opp,TEAM_ABBREVIATION_opp,TEAM_NAME_opp,GAME_DATE_opp,MATCHUP_opp,WL_opp,FGM_opp,FGA_opp,FG3M_opp,FG3A_opp,FTM_opp,FTA_opp,OREB_opp,DREB_opp,AST_opp,TOV_opp,STL_opp,PTS_opp,HOME_GAME,WIN,point_diff,prev_WIN,prev2_WIN
1,2018-19,1610612737,ATL,Atlanta Hawks,21800007,2018-10-17,ATL @ NYK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,1610612752,NYK,New York Knicks,2018-10-17,NYK vs. ATL,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,-19.0,NaN,NaN
5,2018-19,1610612737,ATL,Atlanta Hawks,21800020,2018-10-19,ATL @ MEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,1610612763,MEM,Memphis Grizzlies,2018-10-19,MEM vs. ATL,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,-14.0,0.0,NaN
9,2018-19,1610612737,ATL,Atlanta Hawks,21800036,2018-10-21,ATL @ CLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,1610612739,CLE,Cleveland Cavaliers,2018-10-21,CLE vs. ATL,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,22.0,0.0,0.0
13,2018-19,1610612737,ATL,Atlanta Hawks,21800052,2018-10-24,ATL vs. DAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,1610612742,DAL,Dallas Mavericks,2018-10-24,DAL @ ATL,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,7.0,1.0,0.0
17,2018-19,1610612737,ATL,Atlanta Hawks,21800076,2018-10-27,ATL vs. CHI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,1610612741,CHI,Chicago Bulls,2018-10-27,CHI @ ATL,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,-12.0,1.0,1.0
21,2018-19,1610612737,ATL,Atlanta Hawks,21800088,2018-10-29,ATL @ PHI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,1610612755,PHI,Philadelphia 76ers,2018-10-29,PHI vs. ATL,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,-21.0,0.0,1.0
25,2018-19,1610612737,ATL,Atlanta Hawks,21800097,2018-10-30,ATL @ CLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,1610612739,CLE,Cleveland Cavaliers,2018-10-30,CLE vs. ATL,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,-22.0,0.0,0.0
29,2018-19,1610612737,ATL,Atlanta Hawks,21800114,2018-11-01,ATL vs. SAC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,1610612758,SAC,Sacramento Kings,2018-11-01,SAC @ ATL,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,-31.0,0.0,0.0
33,2018-19,1610612737,ATL,Atlanta Hawks,21800128,2018-11-03,ATL vs. MIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,1610612748,MIA,Miami Heat,2018-11-03,MIA @ ATL,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,5.0,0.0,0.0
37,2018-19,1610612737,ATL,Atlanta Hawks,21800149,2018-11-06,ATL @ CHA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,1610612766,CHA,Charlotte Hornets,2018-11-06,CHA vs. ATL,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,-11.0,1.0,0.0


In [36]:
def final_preparation(df):
       
    # Merge the data once more, since we have team A's data, Team A's opponent's data, Team B's data ,Team B's opponent's data  
    final_merged_df = pd.merge(df, df, on='GAME_ID')

    # Filter out non-sense rows where the same team is playing itself
    final_merged_df = final_merged_df.loc[final_merged_df['TEAM_ABBREVIATION_x'] != final_merged_df['TEAM_ABBREVIATION_y']]

    # Only take the home games (This will include the entire season, as one team is always at home)
    # It just makes it so the home team is always the first set of stats
    final_merged_df = final_merged_df.loc[final_merged_df['HOME_GAME_x'] == 1]

    # Resort the data in chronological order
    final_merged_df = final_merged_df.sort_values(['GAME_DATE_x'])

    # For readability, rename columns    
    new_col_names = ['HOME_' + x[:-2] if x[-2:] == '_x' else 'AWAY_' + x[:-2] for x in final_merged_df.columns.to_list()]
    final_merged_df.columns = new_col_names
    
    final_merged_df.rename(columns={'HOME_point_diff':'point_diff'}, inplace=True)
    
    # Drop unnecessary columns
    cols_to_drop = ['HOME_TEAM_ID', 'HOME_TEAM_NAME', 'HOME_MATCHUP',
                    'HOME_SEASON_YEAR_opp', 'HOME_GAME_DATE',
                    'HOME_TEAM_ID_opp', 'HOME_TEAM_ABBREVIATION_opp', 
                    'HOME_TEAM_NAME_opp', 'HOME_GAME_DATE_opp',
                    'HOME_MATCHUP_opp', 'HOME_WL_opp',
                    'HOME_HOME_GAME', 'AWAY_GAME_', 'AWAY_SEASON_YEAR',
                    'AWAY_TEAM_ID', 'AWAY_MATCHUP', 'AWAY_TEAM_NAME',
                    'AWAY_GAME_DATE', 'AWAY_HOME_GAME', 'AWAY_SEASON_YEAR_opp',
                   'AWAY_TEAM_ID_opp', 'AWAY_TEAM_ABBREVIATION_opp',
                   'AWAY_TEAM_NAME_opp', 'AWAY_GAME_DATE_opp',
                   'AWAY_MATCHUP_opp', 'AWAY_WL_opp', 'AWAY_point_diff']
    final_merged_df = final_merged_df.drop(columns=cols_to_drop)
    final_merged_df.dropna(inplace=True)
    return final_merged_df

final_merged_df_2018 = final_preparation(df=weighted_stats_2018)
final_merged_df_2018
                                                  
                               
                                                  
                                                

,HOME_SEASON_YEAR,HOME_TEAM_ABBREVIATION,HOME_MIN,HOME_FGM,HOME_FGA,HOME_FG3M,HOME_FG3A,HOME_FTM,HOME_FTA,HOME_OREB,HOME_DREB,HOME_AST,HOME_TOV,HOME_STL,HOME_BLK,HOME_BLKA,HOME_PF,HOME_PFD,HOME_PTS,HOME_PLUS_MINUS,HOME_FGM_opp,HOME_FGA_opp,HOME_FG3M_opp,HOME_FG3A_opp,HOME_FTM_opp,HOME_FTA_opp,HOME_OREB_opp,HOME_DREB_opp,HOME_AST_opp,HOME_TOV_opp,HOME_STL_opp,HOME_PTS_opp,HOME_WIN,point_diff,HOME_prev_WIN,HOME_prev2_WIN,AWAY_TEAM_ABBREVIATION,AWAY_MIN,AWAY_FGM,AWAY_FGA,AWAY_FG3M,AWAY_FG3A,AWAY_FTM,AWAY_FTA,AWAY_OREB,AWAY_DREB,AWAY_AST,AWAY_TOV,AWAY_STL,AWAY_BLK,AWAY_BLKA,AWAY_PF,AWAY_PFD,AWAY_PTS,AWAY_PLUS_MINUS,AWAY_FGM_opp,AWAY_FGA_opp,AWAY_FG3M_opp,AWAY_FG3A_opp,AWAY_FTM_opp,AWAY_FTA_opp,AWAY_OREB_opp,AWAY_DREB_opp,AWAY_AST_opp,AWAY_TOV_opp,AWAY_STL_opp,AWAY_PTS_opp,AWAY_WIN,AWAY_prev_WIN,AWAY_prev2_WIN
1250,2018-19,NYK,48.00,39.75,89.45,9.40,29.35,15.40,19.75,11.75,32.05,20.20,15.50,7.25,3.70,6.65,22.65,20.60,104.30,-4.70,40.15,83.65,10.15,29.10,18.55,25.10,10.70,33.15,22.75,15.20,8.20,109.00,0.30,-1.0,0.0,1.0,CHI,48.75,37.95,84.80,11.50,30.05,16.95,21.35,8.05,33.20,23.75,15.40,7.35,5.25,7.15,20.45,18.55,104.35,-7.50,41.15,90.10,12.45,35.80,17.10,21.65,11.40,36.05,24.30,14.30,9.00,111.85,0.15,0.0,0.0
3005,2018-19,IND,48.00,39.55,83.25,8.35,24.50,16.60,23.00,7.85,31.30,21.45,13.35,9.10,5.00,6.00,17.20,20.85,104.05,1.10,37.80,83.60,13.00,35.30,14.35,16.70,9.70,32.85,23.40,16.15,6.15,102.95,0.60,-6.0,0.0,1.0,PHI,48.25,37.20,84.25,10.60,31.20,26.20,32.75,10.45,38.60,24.50,18.50,6.95,5.10,5.15,24.70,25.95,111.20,-1.35,41.40,94.40,9.30,30.35,20.45,26.55,10.05,33.50,22.70,13.40,8.00,112.55,0.60,0.0,1.0
1254,2018-19,NOP,48.00,41.90,88.55,9.15,27.75,22.30,29.70,10.85,34.55,25.35,16.30,6.95,5.15,4.45,24.95,23.75,115.25,-6.45,44.50,91.70,12.10,30.65,20.60,26.55,9.00,34.40,27.80,13.30,9.15,121.70,0.20,9.0,0.0,0.0,CHI,50.50,39.15,86.35,10.65,29.75,18.15,23.40,8.90,34.30,22.20,16.25,7.70,5.90,6.75,20.85,20.30,107.10,-4.45,40.50,92.90,11.55,34.20,19.00,23.25,13.15,36.40,23.05,14.90,8.40,111.55,0.35,1.0,0.0
3405,2018-19,LAL,48.25,44.60,90.70,9.35,26.45,17.10,23.55,9.10,34.30,24.35,16.55,8.75,6.30,5.55,22.50,21.65,115.65,-2.20,45.15,95.05,9.85,30.30,17.70,22.25,13.00,34.65,24.90,16.05,8.10,117.85,0.45,4.0,0.0,1.0,MIN,48.00,39.05,93.30,9.90,30.90,19.15,24.20,11.10,30.35,21.85,11.60,9.35,6.75,5.60,20.10,22.40,107.15,-11.20,44.05,91.60,12.40,32.95,17.85,22.65,13.90,38.25,25.95,15.45,6.00,118.35,0.30,0.0,0.0
1778,2018-19,UTA,48.00,38.65,83.80,10.40,31.65,21.20,30.35,10.50,31.70,24.65,14.35,8.25,5.35,3.45,22.25,26.50,108.90,-3.25,42.65,85.75,10.75,28.70,16.10,22.35,8.45,33.50,22.85,14.35,8.35,112.15,0.30,15.0,0.0,0.0,DAL,48.50,40.40,87.35,12.40,33.20,18.30,23.50,10.50,33.75,23.35,17.90,6.05,4.50,4.75,22.25,24.60,111.50,-1.35,41.85,85.60,10.80,27.20,18.35,23.20,7.65,32.25,25.45,14.75,10.20,112.85,0.30,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2289,2018-19,DEN,48.00,40.20,89.50,9.60,29.55,15.30,20.95,9.40,32.05,25.40,11.70,8.80,3.65,5.20,21.35,19.35,105.30,-3.30,40.95,86.75,9.35,28.80,17.35,22.75,11.90,36.40,23.90,14.35,6.90,108.60,0.35,4.0,0.0,0.0,MIN,48.25,42.00,89.05,10.75,29.60,17.90,22.55,7.50,31.85,25.85,12.90,8.50,5.10,5.15,19.55,21.85,112.65,-6.50,45.40,91.45,13.15,34.30,15.20,19.90,11.35,37.35,26.60,15.15,7.30,119.15,0.35,0.0,0.0
4025,2018-19,MIL,48.50,44.10,92.80,13.35,38.05,21.90,28.15,9.10,43.25,25.55,13.20,7.60,6.25,4.50,19.85,21.90,123.45,6.65,42.65,97.85,16.90,43.40,14.60,20.80,10.25,37.40,26.15,12.45,5.55,116.80,0.70,-11.0,1.0,0.0,OKC,48.25,43.40,96.15,13.20,36.95,15.35,21.10,15.60,33.35,24.55,12.70,7.50,4.35,4.90,21.20,22.05,115.35,3.05,40.45,85.40,14.50,38.05,16.90,21.10,7.30,33.70,23.70,14.55,7.35,112.30,0.70,1.0,1.0
2754,2018-19,MEM,49.00,38.30,87.20,12.25,33.40,19.40,24.35,10.30,35

In [37]:
corr = final_merged_df_2018.select_dtypes('number').corr()

correlated_features = []
for i in range(len(corr.columns)):
    for j in range(i):
        if abs(corr.iloc[i, j]) >= 0.9:
            correlated_features.append(corr.columns[i])
            correlated_features.append(corr.columns[j])
print("Correlated Features:", list(correlated_features))
print("Num Features Removed ", len(correlated_features))

Correlated Features: ['HOME_FTA', 'HOME_FTM', 'HOME_FTA_opp', 'HOME_FTM_opp', 'AWAY_FTA', 'AWAY_FTM', 'AWAY_FTA_opp', 'AWAY_FTM_opp']
Num Features Removed  8


## Feature Selection

In [ ]:
import seaborn as sns
from sklearn.feature_selection import VarianceThreshold

corr = final_merged_df_2018.drop(columns=['HOME_SEASON_YEAR', 'HOME_TEAM_ABBREVIATION', 'AWAY_TEAM_ABBREVIATION']).corr()

mask = np.triu(np.ones_like(corr, dtype=bool))

fig, ax = plt.subplots(figsize=(50,50))

sns.heatmap(corr, mask=mask, square=True, linewidths=0.5)

plt.savefig('correlations.jpg')


In [ ]:
def remove_correlated_features(final_merged_df, threshold=1):
    """Removes correlated features from dataset
    INPUTS:
    final_merged_df
    threshold: float between 0 and 1 indicating how highly correlated features
    must be to be removed
    OUTPUT: final_merged_df with the correlated feature columns removed
    """
    corr = final_merged_df.select_dtypes('number').corr()

    correlated_features = set()
    for i in range(len(corr.columns)):
        for j in range(i):
            if abs(corr.iloc[i, j]) >= threshold:
                colname = corr.columns[i]
                correlated_features.add(colname)
    print("Correlated Features:", list(correlated_features))
    print("Num Features Removed ", len(correlated_features))
    final_merged_df.drop(columns=list(correlated_features), inplace=True)
    return final_merged_df

In [ ]:
# Remove perfectly correlated features
final_merged_df = remove_correlated_features(final_merged_df_2018)
final_merged_df

In [ ]:
seasons = ['20{}-{}'.format(x, x+1) for x in range(14, 21)]

gamelogs_2014_2020 = []

for season in seasons:
    df = pd.read_csv('./data/gamelog_{}.csv'.format(season))
    df = clean_team_gamelog(df)
    df = get_cumulative_stats(df)
    df = final_preparation(df)
#     df = remove_correlated_features(df)
    gamelogs_2010_2020.append(df)
    
gamelogs_all_seasons = pd.concat(gamelogs_2010_2020)
    

In [ ]:
gamelogs_all_seasons.to_csv('./data/gamelogs_all_seasons.csv', index=False)

In [ ]:
gamelogs_all_seasons = pd.read_csv('./data/gamelogs_all_seasons.csv')
gamelogs_all_seasons

In [ ]:
seasons = ['2014-15', '2015-16', '2016-17', '2017-18', '2018-19', '2019-20']
gamelogs_2015_20 = gamelogs_all_seasons.loc[gamelogs_all_seasons['HOME_SEASON_YEAR'].isin(seasons)]

In [ ]:
gamelogs_all_seasons = gamelogs_all_seasons.dropna(axis=1)

X_train = gamelogs_all_seasons.loc[gamelogs_all_seasons['HOME_GAME_DATE'] <= '2018-04-10']
X_train.drop(columns = ['HOME_SEASON_YEAR', 'HOME_TEAM_ABBREVIATION', 'HOME_GAME_DATE',
                       'AWAY_TEAM_ABBREVIATION', 'AWAY_SEASON_YEAR_opp', 'AWAY_TEAM_ID_opp',
                       'AWAY_TEAM_ABBREVIATION_opp', 'AWAY_GAME_DATE_opp', 'AWAY_MATCHUP_opp',
                       'AWAY_WL_opp', 'HOME_SEASON_YEAR_opp', 'HOME_TEAM_ID_opp', 
                        'HOME_TEAM_ABBREVIATION_opp', 'HOME_TEAM_NAME_opp', 'HOME_GAME_DATE_opp', 
                        'HOME_MATCHUP_opp', 'HOME_WL_opp', 'AWAY_TEAM_NAME_opp', 'point_diff'],inplace=True)


y_train = gamelogs_all_seasons.loc[gamelogs_all_seasons['HOME_GAME_DATE'] <= '2018-04-10', 'point_diff']


X_test = gamelogs_all_seasons.loc[gamelogs_all_seasons['HOME_GAME_DATE'] > '2018-04-10']
X_test.drop(columns = ['HOME_SEASON_YEAR', 'HOME_TEAM_ABBREVIATION', 'HOME_GAME_DATE',
                       'AWAY_TEAM_ABBREVIATION', 'AWAY_SEASON_YEAR_opp', 'AWAY_TEAM_ID_opp',
                       'AWAY_TEAM_ABBREVIATION_opp', 'AWAY_GAME_DATE_opp', 'AWAY_MATCHUP_opp',
                       'AWAY_WL_opp', 'HOME_SEASON_YEAR_opp', 'HOME_TEAM_ID_opp', 
                        'HOME_TEAM_ABBREVIATION_opp', 'HOME_TEAM_NAME_opp', 'HOME_GAME_DATE_opp', 
                        'HOME_MATCHUP_opp', 'HOME_WL_opp', 'AWAY_TEAM_NAME_opp', 'point_diff'],inplace=True)


y_test = gamelogs_all_seasons.loc[gamelogs_all_seasons['HOME_GAME_DATE'] > '2018-04-10', 'point_diff'] 



    
    

In [ ]:
print(X_train.shape, y_train.shape)

In [ ]:
pd.options.display.max_columns=170

In [ ]:
X_train

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, ElasticNet, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import learning_curve
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import numpy as np
from sklearn.svm import SVR

In [ ]:
model = LinearRegression()
pipeline = Pipeline([('normalize', Normalizer()),
                    ('est', model)
                    ]
                   )


train_sizes = [1, 100, 500, 2000, 5000, 7161]

train_sizes, train_scores, validation_scores = learning_curve(estimator = pipeline,
                                                             X = X_train,
                                                             y = y_train,
                                                             train_sizes = train_sizes,
                                                             scoring='neg_mean_absolute_error')

plt.style.use('fivethirtyeight')

plt.plot(train_sizes, train_scores, label='Training Error')
plt.plot(train_sizes, validation_scores, label='Validation Error')
plt.ylabel('Mean Abs Error', fontsize=14)
plt.xlabel('Training Set Size', fontsize=14)
plt.title('Learning Curve')
plt.legend()
plt.show()

In [ ]:
model = LinearRegression()
pipeline = Pipeline([('normalize', Normalizer()),
                    ('est', model)
                    ]
                   )
pipeline.fit(X_train, y_train)

y_preds = pipeline.predict(X_test)

mae = mean_absolute_error(y_test, y_preds)
print("{} Mean Absolute Error:".format(str(model)), mae)

In [ ]:
coefficients = pipeline.named_steps['est'].coef_

sorted(list(zip(X_train.columns, coefficients)), key=lambda x: x[1], reverse=True)



In [ ]:
pipeline = Pipeline([('normalize', Normalizer()),
                    ('est', Ridge())
                    ]
                   )

params = {'est__alpha' : [0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10]}

gs_ridge = GridSearchCV(pipeline, param_grid = params)

gs_ridge.fit(X_train, y_train)

y_preds = gs_ridge.predict(X_test)

mae = mean_absolute_error(y_test, y_preds)
print("Ridge Regression: Mean Absolute Error:", mae)

In [ ]:
best_ridge = gs_ridge.best_estimator_
best_ridge

ridge = Ridge(alpha=0.1)
ridge.fit(X_train, y_train)
y_preds = ridge.predict(X_test)

mae = mean_absolute_error(y_test, y_preds)
print("Ridge Regression: Mean Absolute Error:", mae)

coefficients = ridge.coef_

sorted(list(zip(X_train.columns, coefficients)), key=lambda x: x[1], reverse=True)

In [ ]:
pipeline = Pipeline([('normalize', Normalizer()),
                    ('est', Lasso())
                    ]
                   )

params = {'est__alpha' : [0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10]}

gs_lasso = GridSearchCV(pipeline, param_grid = params)

gs_lasso.fit(X_train, y_train)

y_preds = gs_lasso.predict(X_test)

mae = mean_absolute_error(y_test, y_preds)
print("Lasso Regression: Mean Absolute Error:", mae)

In [ ]:
gs_lasso.best_params_

In [ ]:
lasso = Lasso(alpha=0.001)
lasso.fit(X_train, y_train)

coefficients = lasso.coef_

sorted(list(zip(X_train.columns, coefficients)), key=lambda x: x[1])

In [ ]:
from xgboost import XGBRegressor
from xgboost import plot_importance

In [ ]:
xgbr = XGBRegressor()

xgbr.fit(X_train, y_train)

y_preds = xgbr.predict(X_test)

mae = mean_absolute_error(y_test, y_preds)

print("XGB Mean Absolute Error:", mae)

In [ ]:
params = {'eta':np.linspace(0.1, 0.3, 3),
         'min_child_weight':[1,3, 5],
         'max_depth':[3, 6, 9],
         'gamma':[0, 0.01, 0.1],
         'subsample':[0.5, 0.7, 1],
         'colsample_bytree':[0.5, 0.7, 1]
         }

xgb_gs = GridSearchCV(estimator=XGBRegressor(),
                     param_grid=params,
                      scoring='neg_mean_absolute_error'
                     )

xgb_gs.fit(X_train, y_train)

y_preds = xgb_gs.predict(X_test)

mae = mean_absolute_error(y_test, y_preds)

print("XGB Mean Absolute Error:", mae)

In [ ]:
best_xgb = xgb_gs.best_estimator_

y_preds = xgb_gs.predict(X_test)

mae = mean_absolute_error(y_test, y_preds)

print("XGB Mean Absolute Error:", mae)

In [ ]:
print(xgb_gs.best_params_)

In [ ]:
sorted(list(zip(X_train.columns, best_xgb.feature_importances_)), key=lambda x: x[1], reverse=True)

In [ ]:
fig, ax = plt.subplots(figsize=(8,20))
plot_importance(best_xgb, ax=ax)

In [ ]:
rf = RandomForestRegressor()

rf.fit(X_train, y_train)

y_preds = rf.predict(X_test)

mae = mean_absolute_error(y_test, y_preds)

print("Random Forest Mean Absolute Error", mae)

In [ ]:
best_xgb.feature_importances_

## Model with Weighted Stats (see get_weighted_cume_stats function)

In [38]:
seasons = ['20{}-{}'.format(x, x+1) for x in range(14, 21)]

gamelogs_2014_2020 = []

for season in seasons:
    df = pd.read_csv('./data/gamelog_{}.csv'.format(season))
    df = clean_team_gamelog(df)
    df = get_weighted_cume_stats(df, n=10)
    df = final_preparation(df)
#     df = remove_correlated_features(df, 0.8)
    gamelogs_2014_2020.append(df)
    
gamelogs_2014_20_weighted = pd.concat(gamelogs_2014_2020)
    

In [42]:
gamelogs_2014_20_weighted.to_csv('./data/gamelogs_all_seasons_weighted.csv')

In [43]:
gamelogs_2014_20_weighted

,HOME_SEASON_YEAR,HOME_TEAM_ABBREVIATION,HOME_MIN,HOME_FGM,HOME_FGA,HOME_FG3M,HOME_FG3A,HOME_FTM,HOME_FTA,HOME_OREB,HOME_DREB,HOME_AST,HOME_TOV,HOME_STL,HOME_BLK,HOME_BLKA,HOME_PF,HOME_PFD,HOME_PTS,HOME_PLUS_MINUS,HOME_FGM_opp,HOME_FGA_opp,HOME_FG3M_opp,HOME_FG3A_opp,HOME_FTM_opp,HOME_FTA_opp,HOME_OREB_opp,HOME_DREB_opp,HOME_AST_opp,HOME_TOV_opp,HOME_STL_opp,HOME_PTS_opp,HOME_WIN,point_diff,HOME_prev_WIN,HOME_prev2_WIN,AWAY_TEAM_ABBREVIATION,AWAY_MIN,AWAY_FGM,AWAY_FGA,AWAY_FG3M,AWAY_FG3A,AWAY_FTM,AWAY_FTA,AWAY_OREB,AWAY_DREB,AWAY_AST,AWAY_TOV,AWAY_STL,AWAY_BLK,AWAY_BLKA,AWAY_PF,AWAY_PFD,AWAY_PTS,AWAY_PLUS_MINUS,AWAY_FGM_opp,AWAY_FGA_opp,AWAY_FG3M_opp,AWAY_FG3A_opp,AWAY_FTM_opp,AWAY_FTA_opp,AWAY_OREB_opp,AWAY_DREB_opp,AWAY_AST_opp,AWAY_TOV_opp,AWAY_STL_opp,AWAY_PTS_opp,AWAY_WIN,AWAY_prev_WIN,AWAY_prev2_WIN
2794,2014-15,MEM,48.00,35.60,80.60,5.30,13.85,22.00,27.25,11.75,29.05,20.25,11.85,8.15,3.05,4.45,19.75,21.90,98.50,4.80,35.80,80.50,7.70,20.40,14.40,19.65,11.55,30.85,16.90,15.65,6.40,93.70,0.90,26.0,1.0,1.0,HOU,48.00,32.05,78.70,10.80,33.10,19.35,29.20,13.85,31.70,18.00,18.35,9.25,6.15,6.80,23.85,23.85,94.25,7.75,31.40,79.45,5.75,20.10,17.95,23.95,11.85,30.35,18.00,18.20,8.45,86.50,0.90,1.0,1.0
977,2014-15,CHA,49.25,36.15,83.65,6.00,17.05,18.60,25.25,10.30,32.70,19.60,13.45,5.55,3.70,3.80,17.80,24.30,96.90,-5.20,39.30,85.40,8.05,23.45,15.45,18.85,9.75,33.70,24.15,13.30,6.30,102.10,0.40,-27.0,0.0,1.0,DAL,48.00,42.00,85.60,8.85,26.25,18.85,23.05,10.70,29.45,25.05,11.05,8.20,6.50,3.30,22.15,24.55,111.70,12.60,35.75,78.85,9.10,24.05,18.50,25.25,12.60,30.85,21.10,20.45,5.00,99.10,0.75,1.0,1.0
2321,2014-15,DET,48.75,35.55,83.60,8.65,23.50,14.10,21.35,11.40,34.35,17.80,13.30,5.30,5.00,5.30,21.50,20.35,93.85,-3.10,36.30,84.65,6.15,19.55,18.20,24.35,11.80,33.95,21.80,11.05,7.20,96.95,0.30,-14.0,0.0,1.0,ORL,48.50,37.60,79.25,7.40,17.25,15.20,21.25,8.20,32.90,20.10,16.00,5.85,3.15,5.65,21.90,21.10,97.80,0.10,37.85,82.75,5.10,17.30,16.90,21.40,8.85,31.60,21.90,13.10,6.90,97.70,0.45,0.0,1.0
4430,2014-15,UTA,48.00,35.50,76.05,7.25,21.80,17.85,23.90,9.65,29.20,22.60,14.70,6.45,5.20,3.65,19.15,19.60,96.10,-6.80,39.70,83.85,7.35,19.15,16.15,20.45,11.55,29.25,18.30,11.00,7.80,102.90,0.35,17.0,0.0,1.0,OKC,48.75,32.45,79.30,7.60,23.35,15.25,21.05,10.05,33.05,18.60,15.50,5.10,6.40,4.60,22.15,21.05,87.75,-1.45,33.40,82.60,7.85,27.80,14.55,21.35,13.05,31.60,19.50,13.30,7.10,89.20,0.30,0.0,0.0
3913,2014-15,MIL,48.25,35.55,80.90,4.65,18.05,14.50,20.10,10.30,32.40,20.05,16.65,8.95,4.45,4.80,23.65,19.35,90.25,-0.80,32.80,79.55,7.55,20.95,17.90,25.40,10.45,32.05,19.50,16.45,8.30,91.05,0.55,4.0,1.0,0.0,NYK,48.00,38.15,82.00,6.90,17.20,14.30,18.10,10.35,29.40,19.80,13.30,6.30,3.35,3.35,25.15,19.35,97.50,-0.30,33.35,75.35,10.00,23.20,21.10,27.80,9.45,30.20,20.60,14.90,6.85,97.80,0.30,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
650,2020-21,SAC,48.00,42.25,86.00,12.70,32.25,17.95,25.05,8.70,30.85,27.25,13.80,5.90,4.50,4.80,17.85,21.45,115.15,-13.65,47.45,91.50,17.10,39.45,16.80,20.85,10.10,33.25,27.65,12.20,9.10,128.80,0.30,-5.0,0.0,0.0,NOP,48.50,39.90,86.25,9.80,30.35,16.15,22.25,11.45,36.25,22.85,16.00,6.10,3.95,5.35,18.50,20.30,105.75,-5.35,38.65,86.25,16.35,41.55,17.45,21.85,8.70,32.50,28.30,12.30,8.35,111.10,0.20,0.0,0.0
230,2020-21,DAL,48.50,40.35,88.20,14.70,39.80,14.85,20.95,9.40,37.55,23.20,13.00,7.25,6.30,3.35,21.15,19.45,110.25,5.10,38.65,89.90,11.35,36.80,16.50,23.60,11.60,36.20,21.45,13.55,7.60,105.15,0.60,-16.0,0.0,1.0,CHI,49.00,43.85,87.90,15.30,36.80,17.20,21.35,8.15,38.05,26.05,18.70,6.80,4.05,4.35,22.30,20.45,120.20,-1.25,43.15,92.20,15.40,39.95,19.75,26.05,9.60,31.80,25.35,12.85,9.35,121.45,0.30,0.0,0.0
137,2020-21,BOS,48.00,42.65,88.15,12.90,33.30,16.35,20.90,10.75,36.25,23.45,15.45,8.30,5.95,4

In [46]:
training_seasons = ['2014-15', '2015-16', '2016-17', '2017-18', '2018-19']
test_season = ['2019-20', '2020-21']

X_train = gamelogs_2014_20_weighted.loc[gamelogs_2014_20_weighted['HOME_SEASON_YEAR'].isin(training_seasons)]
X_train.drop(columns = ['HOME_SEASON_YEAR', 'HOME_TEAM_ABBREVIATION',
                       'AWAY_TEAM_ABBREVIATION', 'point_diff'], inplace=True)

y_train = gamelogs_2014_20_weighted.loc[gamelogs_2014_20_weighted['HOME_SEASON_YEAR'].isin(training_seasons), 'point_diff']


X_test = gamelogs_2014_20_weighted.loc[gamelogs_2014_20_weighted['HOME_SEASON_YEAR'].isin(test_season)]
X_test.drop(columns = ['HOME_SEASON_YEAR', 'HOME_TEAM_ABBREVIATION',
                       'AWAY_TEAM_ABBREVIATION', 'point_diff'],inplace=True)


y_test = gamelogs_2014_20_weighted.loc[gamelogs_2014_20_weighted['HOME_SEASON_YEAR'].isin(test_season), 'point_diff']



    

## Training Models with Weighted Cumulative Stats

In [ ]:
X_train

In [47]:
from sklearn.linear_model import ElasticNet, Lasso, Ridge
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import Normalizer
from sklearn.metrics import mean_absolute_error

In [ ]:
pipeline = Pipeline([('normalize', Normalizer()),
                    ('est', Lasso())
                    ]
                   )

params = {'est__alpha' : [0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10]}

gs_lasso = GridSearchCV(pipeline, param_grid = params)

gs_lasso.fit(X_train, y_train)

best_lasso = gs_lasso.best_estimator_
print('parameters:', gs_lasso.best_params_)

y_preds = best_lasso.predict(X_test)

mae = mean_absolute_error(y_test, y_preds)
print("Lasso Regression: Mean Absolute Error:", mae)


In [ ]:
coefficients = best_lasso.named_steps['est'].coef_

sorted(list(zip(X_train.columns, coefficients)), key = lambda x: x[1], reverse=True)

In [ ]:
en = ElasticNet()

pipeline = Pipeline([('normalize', Normalizer()),
                    ('est', ElasticNet())
                    ]
                   )

params = {'est__alpha' : [0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10],
         'est__l1_ratio':np.linspace(0.1, 0.9, 10)}

gs_en = GridSearchCV(pipeline, param_grid = params, scoring = 'neg_mean_absolute_error')

gs_en.fit(X_train, y_train)

In [ ]:
best_en = gs_en.best_estimator_
print(gs_en.best_params_)

y_preds = best_en.predict(X_test)
mae = mean_absolute_error(y_test, y_preds)
print("Elastic Net MAE:", mae)

In [48]:
pipeline = Pipeline([('normalize', Normalizer()),
                    ('est', SVR())
                    ]
                   )

params = {'est__kernel':['rbf', 'linear', 'poly'],
         'est__degree':[3,4,5],
         'est__gamma':['scale', 'auto'],
         'est__C':np.logspace(-2, 2, 5)}

gs_svr = GridSearchCV(pipeline, param_grid=params, scoring='neg_mean_absolute_error')

gs_svr.fit(X_train, y_train)



GridSearchCV(estimator=Pipeline(steps=[('normalize', Normalizer()),
                                       ('est', SVR())]),
             param_grid={'est__C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'est__degree': [3, 4, 5],
                         'est__gamma': ['scale', 'auto'],
                         'est__kernel': ['rbf', 'linear', 'poly']},
             scoring='neg_mean_absolute_error')

In [49]:
print(gs_svr.best_params_)
best_svr = gs_svr.best_estimator_
y_preds = best_svr.predict(X_test)
mae = mean_absolute_error(y_test, y_preds)

print('mae:', mae)

{'est__C': 10.0, 'est__degree': 3, 'est__gamma': 'scale', 'est__kernel': 'poly'}
mae: 10.805507729346736


In [ ]:
X_train.columns

In [73]:
from sklearn.ensemble import RandomForestRegressor
max_features = list(np.linspace(0.1, 1, 10))
max_features.append('auto')
params = {'max_depth':[12], 
          'min_samples_split': [0.1],
          'min_samples_leaf':[0.1],
         'max_features': ['auto']}

gs_rf = GridSearchCV(RandomForestRegressor(), param_grid=params, scoring='neg_mean_absolute_error', verbose=2)

gs_rf.fit(X_train, y_train)

print(gs_rf.best_params_)
best_rf = gs_rf.best_estimator_
y_preds = best_rf.predict(X_test)
mae = mean_absolute_error(y_test, y_preds)

print('mae:', mae)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] max_depth=12, max_features=auto, min_samples_leaf=0.1, min_samples_split=0.1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  max_depth=12, max_features=auto, min_samples_leaf=0.1, min_samples_split=0.1, total=   1.9s
[CV] max_depth=12, max_features=auto, min_samples_leaf=0.1, min_samples_split=0.1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s remaining:    0.0s


[CV]  max_depth=12, max_features=auto, min_samples_leaf=0.1, min_samples_split=0.1, total=   1.9s
[CV] max_depth=12, max_features=auto, min_samples_leaf=0.1, min_samples_split=0.1 
[CV]  max_depth=12, max_features=auto, min_samples_leaf=0.1, min_samples_split=0.1, total=   1.8s
[CV] max_depth=12, max_features=auto, min_samples_leaf=0.1, min_samples_split=0.1 
[CV]  max_depth=12, max_features=auto, min_samples_leaf=0.1, min_samples_split=0.1, total=   1.9s
[CV] max_depth=12, max_features=auto, min_samples_leaf=0.1, min_samples_split=0.1 
[CV]  max_depth=12, max_features=auto, min_samples_leaf=0.1, min_samples_split=0.1, total=   1.9s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    9.3s finished


{'max_depth': 12, 'max_features': 'auto', 'min_samples_leaf': 0.1, 'min_samples_split': 0.1}
mae: 10.72567300743439


## Saving the Model

In [74]:
import pickle

model_name = 'rf_model.sav'

with open(model_name, 'wb') as file:
    pickle.dump(best_rf, file)

In [75]:
with open(model_name, 'rb') as file:
    pickled_rf = pickle.load(file)
    
pickled_rf

RandomForestRegressor(max_depth=12, min_samples_leaf=0.1, min_samples_split=0.1)

## Using the Model

In [64]:
X_train.columns

Index(['HOME_MIN', 'HOME_FGM', 'HOME_FGA', 'HOME_FG3M', 'HOME_FG3A',
       'HOME_FTM', 'HOME_FTA', 'HOME_OREB', 'HOME_DREB', 'HOME_AST',
       'HOME_TOV', 'HOME_STL', 'HOME_BLK', 'HOME_BLKA', 'HOME_PF', 'HOME_PFD',
       'HOME_PTS', 'HOME_PLUS_MINUS', 'HOME_FGM_opp', 'HOME_FGA_opp',
       'HOME_FG3M_opp', 'HOME_FG3A_opp', 'HOME_FTM_opp', 'HOME_FTA_opp',
       'HOME_OREB_opp', 'HOME_DREB_opp', 'HOME_AST_opp', 'HOME_TOV_opp',
       'HOME_STL_opp', 'HOME_PTS_opp', 'HOME_WIN', 'HOME_prev_WIN',
       'HOME_prev2_WIN', 'AWAY_MIN', 'AWAY_FGM', 'AWAY_FGA', 'AWAY_FG3M',
       'AWAY_FG3A', 'AWAY_FTM', 'AWAY_FTA', 'AWAY_OREB', 'AWAY_DREB',
       'AWAY_AST', 'AWAY_TOV', 'AWAY_STL', 'AWAY_BLK', 'AWAY_BLKA', 'AWAY_PF',
       'AWAY_PFD', 'AWAY_PTS', 'AWAY_PLUS_MINUS', 'AWAY_FGM_opp',
       'AWAY_FGA_opp', 'AWAY_FG3M_opp', 'AWAY_FG3A_opp', 'AWAY_FTM_opp',
       'AWAY_FTA_opp', 'AWAY_OREB_opp', 'AWAY_DREB_opp', 'AWAY_AST_opp',
       'AWAY_TOV_opp', 'AWAY_STL_opp', 'AWAY_PTS_opp', 'AWAY_WIN

In [65]:
from nba_api.stats.endpoints import teamgamelogs

def prep_inputs_for_model(homeTeamAbbr, awayTeamAbbr, season='2020-21'):
    
    gamelogs = teamgamelogs.TeamGameLogs(season_nullable=season).get_data_frames()[0]
    gamelogs_clean = clean_team_gamelog(gamelogs)
        
    home_gl = gamelogs_clean.loc[gamelogs_clean['TEAM_ABBREVIATION'] == homeTeamAbbr]

    away_gl = gamelogs_clean.loc[gamelogs_clean['TEAM_ABBREVIATION'] == awayTeamAbbr]

    home_stats_weighted = get_weighted_cume_stats(home_gl)
    home_stats_last = home_stats_weighted.iloc[-1].to_frame().T

    away_stats_weighted = get_weighted_cume_stats(away_gl)
    away_stats_last = away_stats_weighted.iloc[-1].to_frame().T
    
    inputs = pd.merge(home_stats_last, away_stats_last, on='SEASON_YEAR')
    # For readability, rename columns    
    new_col_names = ['HOME_' + x[:-2] if x[-2:] == '_x' else 'AWAY_' + x[:-2] for x in inputs.columns.to_list()]
    inputs.columns = new_col_names
    
    columns_needed = ['HOME_MIN', 'HOME_FGM', 'HOME_FGA', 'HOME_FG3M', 'HOME_FG3A',
       'HOME_FTM', 'HOME_FTA', 'HOME_OREB', 'HOME_DREB', 'HOME_AST',
       'HOME_TOV', 'HOME_STL', 'HOME_BLK', 'HOME_BLKA', 'HOME_PF', 'HOME_PFD',
       'HOME_PTS', 'HOME_PLUS_MINUS', 'HOME_FGM_opp', 'HOME_FGA_opp',
       'HOME_FG3M_opp', 'HOME_FG3A_opp', 'HOME_FTM_opp', 'HOME_FTA_opp',
       'HOME_OREB_opp', 'HOME_DREB_opp', 'HOME_AST_opp', 'HOME_TOV_opp',
       'HOME_STL_opp', 'HOME_PTS_opp', 'HOME_WIN', 'HOME_prev_WIN',
       'HOME_prev2_WIN', 'AWAY_MIN', 'AWAY_FGM', 'AWAY_FGA', 'AWAY_FG3M',
       'AWAY_FG3A', 'AWAY_FTM', 'AWAY_FTA', 'AWAY_OREB', 'AWAY_DREB',
       'AWAY_AST', 'AWAY_TOV', 'AWAY_STL', 'AWAY_BLK', 'AWAY_BLKA', 'AWAY_PF',
       'AWAY_PFD', 'AWAY_PTS', 'AWAY_PLUS_MINUS', 'AWAY_FGM_opp',
       'AWAY_FGA_opp', 'AWAY_FG3M_opp', 'AWAY_FG3A_opp', 'AWAY_FTM_opp',
       'AWAY_FTA_opp', 'AWAY_OREB_opp', 'AWAY_DREB_opp', 'AWAY_AST_opp',
       'AWAY_TOV_opp', 'AWAY_STL_opp', 'AWAY_PTS_opp', 'AWAY_WIN',
       'AWAY_prev_WIN', 'AWAY_prev2_WIN']
    inputs = inputs[columns_needed]
    
    return inputs

In [66]:
prep_inputs_for_model('LAC', 'OKC', season='2020-21')

,HOME_MIN,HOME_FGM,HOME_FGA,HOME_FG3M,HOME_FG3A,HOME_FTM,HOME_FTA,HOME_OREB,HOME_DREB,HOME_AST,HOME_TOV,HOME_STL,HOME_BLK,HOME_BLKA,HOME_PF,HOME_PFD,HOME_PTS,HOME_PLUS_MINUS,HOME_FGM_opp,HOME_FGA_opp,HOME_FG3M_opp,HOME_FG3A_opp,HOME_FTM_opp,HOME_FTA_opp,HOME_OREB_opp,HOME_DREB_opp,HOME_AST_opp,HOME_TOV_opp,HOME_STL_opp,HOME_PTS_opp,HOME_WIN,HOME_prev_WIN,HOME_prev2_WIN,AWAY_MIN,AWAY_FGM,AWAY_FGA,AWAY_FG3M,AWAY_FG3A,AWAY_FTM,AWAY_FTA,AWAY_OREB,AWAY_DREB,AWAY_AST,AWAY_TOV,AWAY_STL,AWAY_BLK,AWAY_BLKA,AWAY_PF,AWAY_PFD,AWAY_PTS,AWAY_PLUS_MINUS,AWAY_FGM_opp,AWAY_FGA_opp,AWAY_FG3M_opp,AWAY_FG3A_opp,AWAY_FTM_opp,AWAY_FTA_opp,AWAY_OREB_opp,AWAY_DREB_opp,AWAY_AST_opp,AWAY_TOV_opp,AWAY_STL_opp,AWAY_PTS_opp,AWAY_WIN,AWAY_prev_WIN,AWAY_prev2_WIN
0,48,41.5,88.5,14.1,36.45,17,19.3,9.8,37.85,24.35,12.25,9.05,4.55,3.1,16.9,16.8,114.1,12.5,38.45,87.05,11.2,32.2,13.5,16.8,8.45,34.45,22.65,15,6.95,101.6,0.85,1,1,48.5,40.85,88.95,13.4,38.75,14,20.45,7.65,36.95,22.5,12.6,6.95,4.6,4.15,19.5,18.4,109.1,-3.45,41.7,94.05,14,40.4,15.15,18.75,10.65,39.15,22.65,12.85,6.55,112.55,0.55,1,1


In [67]:
predictions_df = pd.DataFrame({'date':[], 'home_team':[], 'away_team':[], 'prediction':[], 'result':[]})
predictions_df

,date,home_team,away_team,prediction,result


In [68]:
from datetime import datetime
from nba_api.stats.endpoints import scoreboardv2



def get_matchups(date = datetime.today().strftime('%Y-%m-%d')):
    scoreboard_df = scoreboardv2.ScoreboardV2(game_date=datetime.today().strftime("%Y-%m-%d")).get_data_frames()[0]
    teams = scoreboard_df['GAMECODE'].str[-6:]
    away_teams = teams.str[:3]
    home_teams = teams.str[3:]
    matchups = tuple(zip(home_teams, away_teams))
    return matchups

get_matchups()
    
    

(('ATL', 'LAL'),
 ('MIA', 'CHA'),
 ('CLE', 'MIN'),
 ('CHI', 'NYK'),
 ('MIL', 'POR'),
 ('NOP', 'SAC'),
 ('OKC', 'HOU'),
 ('DAL', 'PHX'),
 ('SAS', 'MEM'),
 ('DEN', 'DET'))

In [76]:
matchups = get_matchups()
todays_preds = pd.DataFrame({'model', 'date':[], 'home_team':[], 'away_team':[], 'prediction':[], 'result':[]})
game_date = []
home_team = []
away_team = []
game_prediction = []
result = []
for matchup in matchups:
    date = datetime.today().strftime('%Y-%m-%d')
    inputs = prep_inputs_for_model(matchup[0], matchup[1])
    prediction = round(pickled_rf.predict(inputs)[0], 1)
    game_date.append(date)
    home_team.append(matchup[0])
    away_team.append(matchup[1])
    game_prediction.append(prediction)
    result.append(np.nan)
    row = pd.DataFrame({'date':[date], 'home_team':[matchup[0]], 'away_team':[matchup[1]], 'prediction':[prediction], 'result':[np.nan]})
    todays_preds = pd.concat([todays_preds, row])

todays_preds
    
predictions_df = predictions_df.append(todays_preds)

predictions_df

,date,home_team,away_team,prediction,result
0,2021-01-26,ATL,LAL,1.7,NaN
0,2021-01-26,MIA,CHA,-1.6,NaN
0,2021-01-26,CLE,MIN,3.6,NaN
0,2021-01-26,CHI,NYK,5.8,NaN
0,2021-01-26,MIL,POR,6.9,NaN
0,2021-01-26,NOP,SAC,3.2,NaN
0,2021-01-26,OKC,HOU,-0.8,NaN
0,2021-01-26,DAL,PHX,-2.7,NaN
0,2021-01-26,SAS,MEM,3.8,NaN
0,2021-01-26,DEN,DET,7.6,NaN


In [71]:
predictions_df.to_csv('predictions.csv', index=False)

In [164]:
todays_preds

,date,home_team,away_team,prediction,result
0,2021-01-26,CLE,DET,-0.4,NaN
0,2021-01-26,ORL,SAC,4.0,NaN
0,2021-01-26,TOR,MIL,4.7,NaN
0,2021-01-26,CHA,IND,3.7,NaN
0,2021-01-26,PHI,LAL,-0.2,NaN
0,2021-01-26,ATL,BKN,1.1,NaN
0,2021-01-26,DEN,MIA,6.2,NaN
0,2021-01-26,SAS,BOS,1.9,NaN
0,2021-01-26,UTA,DAL,11.8,NaN
